# A minimal example of creating a custom LJ potential in OpenMM
This notebook shows an example of moving the LJ potential from `NonbondedForce` to `CustomNonbondedForce` for a butane molecule in vacuum. The parameters for the molecule are obtained from GAFF.

In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
import simtk.unit as unit

## Lennard-Jones energy expression

In [2]:
# 12-6 LJ
LJ_potential = "4*epsilon*((sigma/r)^12-(sigma/r)^6); "

# Combining rules
Lorentz_Berthelot = "sigma=0.5*(sigma1+sigma2); epsilon=sqrt(epsilon1*epsilon2)"
Good_Hope = "sigma=sqrt(sigma1*sigma2); epsilon=sqrt(epsilon1*epsilon2)"
Fender_Halsey = "sigma=0.5*(sigma1+sigma2); epsilon=2*epsilon1*epsilon2/(epsilon1+epsilon2)"
Waldman_Hagler = "sigma=(0.5*(sigma1^6+sigma2^6))^(1/6); epsilon=sqrt(epsilon1*epsilon2)*(2*sigma1^3*sigma2^3)/(sigma1^6+sigma2^6)"

## Load Amber topology and create OpenMM System

In [3]:
prmtop = AmberPrmtopFile('butane.prmtop')
inpcrd = AmberInpcrdFile('butane.rst7')

In [4]:
system = prmtop.createSystem(
    nonbondedMethod=NoCutoff,
#     nonbondedCutoff=9.0 * unit.angstrom,
    constraints=HBonds,
)
forces = {system.getForce(index).__class__.__name__: system.getForce(index) for index in range(system.getNumForces())}
nonbonded_force = forces['NonbondedForce']
# nonbonded_force.setUseDispersionCorrection(False)

## Calculate the potential energy with the current LJ

In [5]:
integrator = LangevinIntegrator(298.15 * unit.kelvin, 1.0 / unit.picoseconds, 2.0 * unit.femtosecond)
simulation = Simulation(
    prmtop.topology, 
    system, 
    integrator,
    openmm.Platform.getPlatformByName('CPU'),
)
simulation.context.setPositions(inpcrd.positions)

In [6]:
total_energy = simulation.context.getState(getEnergy=True, groups={0})
energy_original = total_energy.getPotentialEnergy() / unit.kilocalorie_per_mole

## Create new CustomNonbondedForce
Example here is to replace the 12-6 LJ potential in `NonbondedForce` and move it to `CustomNonbondedForce`.

In [7]:
LJ_new = CustomNonbondedForce(LJ_potential + Lorentz_Berthelot)
LJ_new.addPerParticleParameter("sigma")
LJ_new.addPerParticleParameter("epsilon")
LJ_new.setNonbondedMethod(CustomNonbondedForce.CutoffPeriodic)
# LJ_new.setUseLongRangeCorrection(nonbonded_force.getUseDispersionCorrection())

In [8]:
# Set the LJ parameters
for atom in range(system.getNumParticles()):
    current_parameters = nonbonded_force.getParticleParameters(atom)
    # charge: current_parameters[0] | sigma: current_parameters[1] | epsilon: current_parameters[2]
    LJ_new.addParticle([current_parameters[1], current_parameters[2]])

In [9]:
# Set exclusions
for index in range(nonbonded_force.getNumExceptions()):
    [atomi, atomj, chargeprod, sigma, epsilon]  = nonbonded_force.getExceptionParameters(index)
    LJ_new.addExclusion(atomi, atomj)

In [10]:
# Add new LJ potential to OpenMM system object
system.addForce(LJ_new)

5

## Set LJ parameters in `NonbondedForce` to Zero
Since we only want `CustomNonbondedForce` to take care of the LJ potential

In [11]:
for atom in range(system.getNumParticles()):
    current_parameters = nonbonded_force.getParticleParameters(atom)
    nonbonded_force.setParticleParameters(atom, current_parameters[0], 0.0, 0.0)

## Calculate the energy with the new LJ

In [12]:
integrator = LangevinIntegrator(298.15 * unit.kelvin, 1.0 / unit.picoseconds, 2.0 * unit.femtosecond)
simulation = Simulation(
    prmtop.topology, 
    system, 
    integrator,
    openmm.Platform.getPlatformByName('CPU'),
)
simulation.context.setPositions(inpcrd.positions)

In [13]:
total_energy = simulation.context.getState(getEnergy=True, groups={0})
energy_custom = total_energy.getPotentialEnergy() / unit.kilocalorie_per_mole

## Compare Energies

In [14]:
print("Total energy of butane:")
print(f" - Original = {energy_original:.5f} kcal/mol")
print(f" - Custom   = {energy_custom:.5f} kcal/mol")

Total energy of butane:
 - Original = 3.05833 kcal/mol
 - Custom   = 3.05833 kcal/mol
